# Fixture Difficulty Rating Rotation Optimization

In FPL, Fixture Difficulty Rating (FDR) is a well-known metric. Simply put, FDR shows how difficult a team's game in every gameweek.

Finding pairs of teams that we can rotate is a common approach to simplify the FPL as a problem. Here, I will show how an FDR model can be written to find pairs, triplets, even under special conditions (e.g. FreeHit)

In [1]:
import pandas as pd
import sasoptpy as so
import requests
import json
import os
import random
from subprocess import Popen
from IPython.display import display, HTML
from math import exp

In [2]:
ratings = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_global_rankings.csv")
ratings.head()

,rank,prev_rank,name,league,off,def,spi
0,1,1.0,Manchester City,Barclays Premier League,2.79,0.25,92.42
1,2,2.0,Bayern Munich,German Bundesliga,3.30,0.60,90.65
2,3,3.0,Chelsea,Barclays Premier League,2.42,0.20,90.53
3,4,4.0,Barcelona,Spanish Primera Division,3.14,0.56,90.10
4,5,5.0,Liverpool,Barclays Premier League,2.69,0.40,89.17


In [3]:
hfa = 0.15
fixture = pd.read_excel("../data/ben_2021_22.xlsx", sheet_name="HA Schedule", header=2, index_col=0, usecols=range(1, 41)).drop(columns=["Unnamed: 2"])
fixture.index.name ='team'
fixture_original = fixture.copy()
# fixture = fixture.applymap(lambda x: x.upper())
# fixture.index = fixture.index.str.upper()
fix_dict = fixture.to_dict('index')
fixture.head()

,1,2,3,4,5,6,7,8,9,10,...,29,30,31,32,33,34,35,36,37,38
team,,,,,,,,,,,,,,,,,,,,,
ARS,bre,CHE,mci,NOR,bur,TOT,bha,CRY,AVL,lei,...,LEI,avl,cry,BHA,sou,MUN,whu,LEE,new,EVE
AVL,wat,NEW,BRE,che,EVE,mun,tot,WOL,ars,WHU,...,whu,ARS,wol,TOT,LIV,lei,NOR,bur,CRY,mci
BRE,ARS,cry,avl,BHA,wol,LIV,whu,CHE,LEI,bur,...,BUR,lei,che,WHU,wat,TOT,mun,SOU,eve,LEE
BHA,bur,WAT,EVE,bre,LEI,cry,ARS,nor,MCI,liv,...,LIV,mci,NOR,ars,tot,SOU,wol,MUN,lee,WHU
BUR,BHA,liv,LEE,eve,ARS,lei,NOR,mci,sou,BRE,...,bre,SOU,MCI,nor,whu,WOL,wat,AVL,tot,NEW


In [4]:
teams = {
    'ARS': {'name': 'Arsenal'},
    'AVL':  {'name': 'Aston Villa'},
    'BRE':  {'name': 'Brentford'},
    'BHA':  {'name': 'Brighton and Hove Albion'},
    'BUR':  {'name': 'Burnley'},
    'CHE':  {'name': 'Chelsea'},
    'CRY':  {'name': 'Crystal Palace'},
    'EVE':  {'name': 'Everton'},
    'LEI':  {'name': 'Leicester City'},
    'LEE': {'name':  'Leeds United'},
    'LIV': {'name':  'Liverpool'},
    'MCI': {'name':  'Manchester City'},
    'MUN': {'name':  'Manchester United'},
    'NEW': {'name':  'Newcastle'},
    'NOR': {'name':  'Norwich City'},
    'SOU': {'name':  'Southampton'},
    'TOT': {'name':  'Tottenham Hotspur'},
    'WAT': {'name':  'Watford'},
    'WHU': {'name':  'West Ham United'},
    'WOL': {'name':  'Wolverhampton'}
}

In [5]:
for team, val in teams.items():
    rating = ratings.loc[ratings.name == val['name'], 'spi'].values[0]
    val['rating'] = rating
teams

{'ARS': {'name': 'Arsenal', 'rating': 83.72},
 'AVL': {'name': 'Aston Villa', 'rating': 72.18},
 'BRE': {'name': 'Brentford', 'rating': 67.01},
 'BHA': {'name': 'Brighton and Hove Albion', 'rating': 78.71},
 'BUR': {'name': 'Burnley', 'rating': 66.79},
 'CHE': {'name': 'Chelsea', 'rating': 90.53},
 'CRY': {'name': 'Crystal Palace', 'rating': 61.55},
 'EVE': {'name': 'Everton', 'rating': 72.28},
 'LEI': {'name': 'Leicester City', 'rating': 78.22},
 'LEE': {'name': 'Leeds United', 'rating': 74.94},
 'LIV': {'name': 'Liverpool', 'rating': 89.17},
 'MCI': {'name': 'Manchester City', 'rating': 92.42},
 'MUN': {'name': 'Manchester United', 'rating': 85.53},
 'NEW': {'name': 'Newcastle', 'rating': 67.79},
 'NOR': {'name': 'Norwich City', 'rating': 65.03},
 'SOU': {'name': 'Southampton', 'rating': 67.01},
 'TOT': {'name': 'Tottenham Hotspur', 'rating': 78.06},
 'WAT': {'name': 'Watford', 'rating': 59.48},
 'WHU': {'name': 'West Ham United', 'rating': 76.0},
 'WOL': {'name': 'Wolverhampton', 'r

In [6]:
# Sets
team_list = list(teams.keys())
gameweeks = list(range(1,39))

In [7]:
fdr = {}
for t in team_list:
    for w in range(1,39):
        try: # HOME
            fdr[t,w] = teams[fix_dict[t][w]]['rating'] / exp(hfa)
        except: # AWAY
            fdr[t,w] = teams[(fix_dict[t][w]).upper()]['rating'] * exp(hfa)

In [8]:
pd.set_option('display.max_columns', None) 

In [9]:
def read_solution(m):
    with open('fdr.sol', 'r') as f:
        for v in m.get_variables():
            v.set_value(0)
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            v = m.get_variable(words[1])
            v.set_value(float(words[2]))

def print_solution(m, gws):
    pick_team = m.get_variable('pick_team')
    pick_team_gw = m.get_variable('pick_team_gw')
    # Print solution
    selected_teams = []
    gameweek_picks = []
    for t in team_list:
        entry = {'team': t}
        if pick_team[t].get_value() > 0:
            selected_teams.append(t)
            for g in gws:
                entry.update({g: round(pick_team_gw[t,g].get_value() * fdr[t,g], 3) })
            gameweek_picks.append(entry)
    
    # Print and first table - values
    print(f'\nSelected: {" and ".join(selected_teams)}. Total FDR: {round(m.get_objective_value(),3)}')
    pick_df = pd.DataFrame(gameweek_picks)
    s = pick_df.style
    colored_vals = lambda x: 'background-color: lightblue; color: black' if type(x) == float and x > 0 else 'color: white'
    s.applymap(colored_vals)
    display(HTML(s.render().replace("000", "")))
    
    # Second table - names
    fr = fixture_original.reset_index()
    selected_fixture = fr[fr['team'].isin(selected_teams)].copy().reset_index(drop=True)
    selected_fixture = selected_fixture[['team'] + gws]
    s2 = selected_fixture.style
    def color_based_on_selection(cell):
        d = cell.copy()
        for c in d.columns:
            if c == 'team': continue
            for r in d.index:
                if pick_df.loc[r, c]:
                    d.loc[r, c] = 'background-color: green; color: white'
                else:
                    d.loc[r, c] = ''
        return d
    s2.apply(color_based_on_selection, axis=None)
    display(HTML(s2.render()))
    return selected_teams

In [10]:
def solve_N_pair_problem(N=2, max_iter=1, first_gw=1, last_gw=38, exclude=[]):
    m = so.Model(name='N_rotation_pairs')
    team_list = list(teams.keys())
    gameweeks = list(range(first_gw, last_gw+1))
    pick_team = m.add_variables(team_list, vartype=so.binary, name='pick_team')
    pick_team_gw = m.add_variables(team_list, gameweeks, vartype=so.binary, name='pick_team_gw')

    if len(exclude) > 0:
        m.add_constraints((pick_team[t] == 0 for t in exclude), name=f'disable_{t}')

    m.add_constraint(so.expr_sum(pick_team[t] for t in team_list) == N, name='pick_2')
    m.add_constraints((so.expr_sum(pick_team_gw[t, g] for t in team_list) == 1 for g in gameweeks), name='pick_1_per_gw')
    m.add_constraints((pick_team_gw[t,g] <= pick_team[t] for t in team_list for g in gameweeks), name='valid_picks_only')

    # Force using each team at least once
    m.add_constraints((so.expr_sum(pick_team_gw[t,g] for g in gameweeks) >= pick_team[t] for t in team_list), name='force_use')

    m.set_objective(so.expr_sum(fdr[t, g] * pick_team_gw[t, g] for t in team_list for g in gameweeks), sense='N', name='total_fdr')

    m.export_mps("fdr.mps")
    command = "cbc fdr.mps solve solu fdr.sol"
    Popen(command).wait()
    read_solution(m)
    selected_teams = print_solution(m, gameweeks)
    for it in range(1, max_iter):
        c = m.add_constraint(so.expr_sum(pick_team[t] for t in selected_teams) <= N-1, name=f'cutoff_{it}')
        m.export_mps("fdr.mps")
        Popen(command).wait()
        read_solution(m)
        selected_teams = print_solution(m, gameweeks)

In [11]:
solve_N_pair_problem(N=2, max_iter=5)

NOTE: Initialized model N_rotation_pairs.

Selected: MCI and MUN. Total FDR: 2422.127


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,MCI,0.,55.972,72.058,0.,57.676,0.,0.,57.487,0.,52.977,0.,62.212,65.414,0.,0.,59.914,64.501,0.,67.325,0.,0.,77.920,0.,57.676,0.,67.187,0.,73.616,0.,67.746,0.,76.749,0.,51.195,0.,58.347,0.,62.126
1,MUN,64.501,0.,0.,58.347,0.,62.126,62.212,0.,76.749,0.,79.547,0.,0.,72.058,52.977,0.,0.,67.746,0.,57.487,59.914,0.,65.414,0.,57.676,0.,51.195,0.,67.187,0.,67.325,0.,55.972,0.,57.676,0.,77.920,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,MCI,tot,NOR,ARS,lei,SOU,che,liv,BUR,bha,CRY,mun,EVE,WHU,avl,wat,WOL,LEE,new,LEI,bre,ars,CHE,sou,BRE,nor,TOT,eve,MUN,cry,BHA,bur,LIV,wol,WAT,lee,NEW,whu,AVL
1,MUN,LEE,sou,wol,NEW,whu,AVL,EVE,lei,LIV,tot,MCI,wat,che,ARS,CRY,nor,bre,BHA,new,BUR,WOL,avl,WHU,bur,SOU,lee,WAT,mci,TOT,liv,LEI,eve,NOR,ars,BRE,bha,CHE,cry



Selected: AVL and CHE. Total FDR: 2422.698


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,AVL,0.,58.347,57.676,0.,62.212,0.,0.,59.914,0.,65.414,0.,67.746,71.511,0.,67.325,0.,0.,57.487,77.920,0.,0.,73.616,0.,64.501,0.,51.195,0.,57.676,0.,72.058,0.,67.187,76.749,0.,55.972,0.,52.977,0.
1,CHE,52.977,0.,0.,62.126,0.,79.547,57.676,0.,55.972,0.,57.487,0.,0.,69.106,0.,64.501,62.212,0.,0.,67.746,76.749,0.,67.187,0.,72.058,0.,67.325,0.,58.347,0.,57.676,0.,0.,65.414,0.,59.914,0.,51.195


...

,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,77.855,77.920,0.,55.972,77.599,67.187,0.,52.977,62.126,0.,51.195,0.,58.347,0.,0.,57.676,65.414,0.,0.,59.914,0.,0.,57.487,0.,0.,57.676,76.749,0.,67.325,0.,0.,67.746,0.,73.616,0.,64.501,0.,62.212
1,TOT,0.,0.,51.195,0.,0.,0.,62.126,0.,0.,73.616,0.,64.501,0.,57.676,55.972,0.,0.,76.749,52.977,0.,69.106,72.058,0.,57.676,59.914,0.,0.,62.212,0.,65.414,58.347,0.,67.746,0.,67.325,0.,57.487,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,bre,CHE,mci,NOR,bur,TOT,bha,CRY,AVL,lei,WAT,liv,NEW,mun,eve,SOU,WHU,lee,nor,WOL,MCI,tot,BUR,wol,che,BRE,LIV,wat,LEI,avl,cry,BHA,sou,MUN,whu,LEE,new,EVE
1,TOT,MCI,wol,WAT,cry,CHE,ars,AVL,new,whu,MUN,eve,LEE,bur,BRE,NOR,bha,lei,LIV,CRY,sou,wat,ARS,che,SOU,WOL,mci,lee,EVE,mun,WHU,NEW,avl,BHA,bre,LEI,liv,BUR,nor



Selected: LIV and MUN. Total FDR: 2428.118


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,LIV,0.,57.487,77.920,0.,52.977,0.,0.,69.106,0.,67.746,0.,0.,57.676,0.,0.,62.126,58.347,0.,64.501,0.,0.,57.676,0.,67.325,0.,55.972,0.,65.414,0.,73.616,51.195,0.,0.,62.212,0.,67.187,77.855,59.914
1,MUN,64.501,0.,0.,58.347,0.,62.126,62.212,0.,76.749,0.,79.547,69.106,0.,72.058,52.977,0.,0.,67.746,0.,57.487,59.914,0.,65.414,0.,57.676,0.,51.195,0.,67.187,0.,0.,83.977,55.972,0.,57.676,0.,0.,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,LIV,nor,BUR,CHE,lee,CRY,bre,MCI,wat,mun,BHA,whu,ARS,SOU,eve,wol,AVL,NEW,tot,LEE,lei,che,BRE,cry,LEI,bur,NOR,ars,WHU,bha,MUN,WAT,mci,avl,EVE,new,TOT,sou,WOL
1,MUN,LEE,sou,wol,NEW,whu,AVL,EVE,lei,LIV,tot,MCI,wat,che,ARS,CRY,nor,bre,BHA,new,BUR,WOL,avl,WHU,bur,SOU,lee,WAT,mci,TOT,liv,LEI,eve,NOR,ars,BRE,bha,CHE,cry


In [12]:
solve_N_pair_problem(N=3, max_iter=3)

NOTE: Initialized model N_rotation_pairs.

Selected: ARS and MUN and WOL. Total FDR: 2299.846


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,0.,0.,0.,55.972,0.,0.,0.,52.977,62.126,0.,51.195,0.,58.347,0.,0.,57.676,65.414,0.,0.,0.,0.,0.,57.487,0.,0.,57.676,0.,0.,0.,0.,0.,67.746,0.,73.616,0.,64.501,0.,62.212
1,MUN,64.501,0.,0.,0.,0.,62.126,0.,0.,0.,0.,0.,0.,0.,0.,52.977,0.,0.,67.746,0.,57.487,59.914,0.,0.,0.,57.676,0.,51.195,0.,67.187,0.,0.,0.,55.972,0.,57.676,0.,0.,0.
2,WOL,0.,67.187,73.616,0.,57.676,0.,58.347,0.,0.,62.212,0.,65.414,0.,57.487,0.,0.,0.,0.,51.195,0.,0.,57.676,0.,72.058,0.,0.,0.,52.977,0.,64.501,62.126,0.,0.,0.,0.,0.,55.972,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,bre,CHE,mci,NOR,bur,TOT,bha,CRY,AVL,lei,WAT,liv,NEW,mun,eve,SOU,WHU,lee,nor,WOL,MCI,tot,BUR,wol,che,BRE,LIV,wat,LEI,avl,cry,BHA,sou,MUN,whu,LEE,new,EVE
1,MUN,LEE,sou,wol,NEW,whu,AVL,EVE,lei,LIV,tot,MCI,wat,che,ARS,CRY,nor,bre,BHA,new,BUR,WOL,avl,WHU,bur,SOU,lee,WAT,mci,TOT,liv,LEI,eve,NOR,ars,BRE,bha,CHE,cry
2,WOL,lei,TOT,MUN,wat,BRE,sou,NEW,avl,lee,EVE,cry,WHU,nor,BUR,LIV,mci,bha,CHE,WAT,ars,mun,SOU,bre,ARS,tot,LEI,whu,CRY,eve,LEE,AVL,new,MCI,bur,BHA,che,NOR,liv



Selected: EVE and LIV and TOT. Total FDR: 2301.516


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,EVE,57.676,0.,0.,57.487,0.,55.972,0.,65.414,51.195,0.,67.187,0.,0.,0.,0.,0.,0.,67.325,0.,58.347,67.746,0.,62.126,0.,0.,0.,79.547,0.,59.914,0.,0.,73.616,52.977,0.,0.,0.,0.,0.
1,LIV,0.,57.487,0.,0.,52.977,0.,0.,0.,0.,67.746,0.,0.,57.676,0.,0.,62.126,58.347,0.,0.,0.,0.,57.676,0.,0.,0.,55.972,0.,0.,0.,0.,51.195,0.,0.,62.212,0.,67.187,0.,59.914
2,TOT,0.,0.,51.195,0.,0.,0.,62.126,0.,0.,0.,0.,64.501,0.,57.676,55.972,0.,0.,0.,52.977,0.,0.,0.,0.,57.676,59.914,0.,0.,62.212,0.,65.414,0.,0.,0.,0.,67.325,0.,57.487,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,EVE,SOU,lee,bha,BUR,avl,NOR,mun,WHU,WAT,wol,TOT,mci,bre,LIV,ARS,cry,che,LEI,bur,NEW,BHA,nor,AVL,new,LEE,sou,MCI,tot,WOL,wat,whu,MUN,CRY,liv,CHE,lei,BRE,ars
1,LIV,nor,BUR,CHE,lee,CRY,bre,MCI,wat,mun,BHA,whu,ARS,SOU,eve,wol,AVL,NEW,tot,LEE,lei,che,BRE,cry,LEI,bur,NOR,ars,WHU,bha,MUN,WAT,mci,avl,EVE,new,TOT,sou,WOL
2,TOT,MCI,wol,WAT,cry,CHE,ars,AVL,new,whu,MUN,eve,LEE,bur,BRE,NOR,bha,lei,LIV,CRY,sou,wat,ARS,che,SOU,WOL,mci,lee,EVE,mun,WHU,NEW,avl,BHA,bre,LEI,liv,BUR,nor



Selected: LIV and SOU and TOT. Total FDR: 2303.19


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,LIV,75.554,57.487,0.,0.,52.977,0.,0.,0.,0.,67.746,0.,0.,57.676,0.,0.,62.126,58.347,0.,0.,0.,0.,57.676,71.511,0.,0.,55.972,0.,0.,0.,0.,51.195,0.,0.,62.212,0.,67.187,0.,59.914
1,SOU,0.,0.,0.,65.414,0.,59.914,0.,64.501,57.487,0.,62.126,0.,0.,0.,0.,0.,0.,57.676,0.,67.187,58.347,0.,0.,0.,0.,0.,55.972,0.,51.195,0.,0.,77.920,0.,0.,52.977,0.,0.,0.
2,TOT,0.,0.,51.195,0.,0.,0.,62.126,0.,0.,0.,0.,64.501,0.,57.676,55.972,0.,0.,0.,52.977,0.,0.,0.,0.,57.676,59.914,0.,0.,62.212,0.,65.414,0.,0.,67.746,0.,0.,0.,57.487,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,LIV,nor,BUR,CHE,lee,CRY,bre,MCI,wat,mun,BHA,whu,ARS,SOU,eve,wol,AVL,NEW,tot,LEE,lei,che,BRE,cry,LEI,bur,NOR,ars,WHU,bha,MUN,WAT,mci,avl,EVE,new,TOT,sou,WOL
1,SOU,eve,MUN,new,WHU,mci,WOL,che,LEE,BUR,wat,AVL,nor,liv,LEI,BHA,ars,cry,BRE,whu,TOT,NEW,wol,MCI,tot,mun,EVE,NOR,avl,WAT,bur,lee,CHE,ARS,bha,CRY,bre,LIV,lei
2,TOT,MCI,wol,WAT,cry,CHE,ars,AVL,new,whu,MUN,eve,LEE,bur,BRE,NOR,bha,lei,LIV,CRY,sou,wat,ARS,che,SOU,WOL,mci,lee,EVE,mun,WHU,NEW,avl,BHA,bre,LEI,liv,BUR,nor


In [13]:
# Free hit version
def solve_N_pair_problem_fh(N=2, max_iter=1):
    m = so.Model(name='N_rotation_pairs')
    team_list = list(teams.keys())
    gameweeks = list(range(1,39))
    pick_team = m.add_variables(team_list, vartype=so.binary, name='pick_team')
    pick_team_gw = m.add_variables(team_list, gameweeks, vartype=so.binary, name='pick_team_gw')
    pick_free_hit = m.add_variables(team_list, gameweeks, vartype=so.binary, name='pick_free_hit')

    m.add_constraint(so.expr_sum(pick_team[t] for t in team_list) == N, name='pick_2')
    m.add_constraints((so.expr_sum(pick_team_gw[t, g] + pick_free_hit[t, g] for t in team_list) == 1 for g in gameweeks), name='pick_1_per_gw')
    m.add_constraints((pick_team_gw[t,g] <= pick_team[t] for t in team_list for g in gameweeks), name='valid_picks_only')
    m.add_constraint((so.expr_sum(pick_free_hit[t, g] for t in team_list for g in gameweeks) <= 1), name='single_free_hit')

    # Force using each team at least once
    m.add_constraints((so.expr_sum(pick_team_gw[t,g] for g in gameweeks) >= pick_team[t] for t in team_list), name='force_use')

    m.set_objective(so.expr_sum(fdr[t, g] * (pick_team_gw[t, g] + pick_free_hit[t,g]) for t in team_list for g in gameweeks), sense='N', name='total_fdr')

    m.export_mps("fdr.mps")
    command = "cbc fdr.mps solve solu fdr.sol"
    Popen(command).wait()
    read_solution(m)

    def print_free_hit_sol():
        selected_teams = []
        gameweek_picks = []
        for t in team_list:
            entry = {'team': t}
            if pick_team[t].get_value() > 0 or so.expr_sum(pick_free_hit[t,g] for g in gameweeks).get_value() > 0:
                selected_teams.append(t)
                for g in gameweeks:
                    entry.update({g: round((pick_team_gw[t,g] + pick_free_hit[t,g]).get_value() * fdr[t,g], 3) })
                gameweek_picks.append(entry)
        print(f'Selected: {" and ".join(selected_teams)}. Total FDR: {round(m.get_objective_value(),3)}')
        pick_df = pd.DataFrame(gameweek_picks)
        s = pick_df.style
        colored_vals = lambda x: 'background-color: lightblue; color: black' if type(x) == float and x > 0 else 'color: white'
        s.applymap(colored_vals)
        display(HTML(s.render().replace("000", "")))

        # Second table - names
        fr = fixture_original.reset_index()
        selected_fixture = fr[fr['team'].isin(selected_teams)].copy().reset_index(drop=True)
        s2 = selected_fixture.style
        def color_based_on_selection(cell):
            d = cell.copy()
            for c in d.columns:
                if c == 'team': continue
                for r in d.index:
                    if pick_df.loc[r, c]:
                        d.loc[r, c] = 'background-color: green; color: white'
                    else:
                        d.loc[r, c] = ''
            return d
        s2.apply(color_based_on_selection, axis=None)
        display(HTML(s2.render()))

        return selected_teams

    selected_teams = print_free_hit_sol()
    for it in range(1, max_iter):
        c = m.add_constraint(so.expr_sum(pick_team[t] for t in selected_teams) <= N-1, name=f'cutoff_{it}')
        m.export_mps("fdr.mps")
        Popen(command).wait()
        read_solution(m)
        selected_teams = print_free_hit_sol()

In [14]:
solve_N_pair_problem_fh(2, 3)

NOTE: Initialized model N_rotation_pairs.
Selected: ARS and MCI and MUN. Total FDR: 2393.775


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,51.195,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
1,MCI,0.,55.972,72.058,0.,57.676,0.,0.,57.487,0.,52.977,0.,62.212,65.414,0.,0.,59.914,64.501,0.,67.325,0.,0.,77.920,0.,57.676,0.,67.187,0.,73.616,0.,67.746,0.,76.749,0.,51.195,0.,58.347,0.,62.126
2,MUN,64.501,0.,0.,58.347,0.,62.126,62.212,0.,76.749,0.,0.,0.,0.,72.058,52.977,0.,0.,67.746,0.,57.487,59.914,0.,65.414,0.,57.676,0.,51.195,0.,67.187,0.,67.325,0.,55.972,0.,57.676,0.,77.920,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,bre,CHE,mci,NOR,bur,TOT,bha,CRY,AVL,lei,WAT,liv,NEW,mun,eve,SOU,WHU,lee,nor,WOL,MCI,tot,BUR,wol,che,BRE,LIV,wat,LEI,avl,cry,BHA,sou,MUN,whu,LEE,new,EVE
1,MCI,tot,NOR,ARS,lei,SOU,che,liv,BUR,bha,CRY,mun,EVE,WHU,avl,wat,WOL,LEE,new,LEI,bre,ars,CHE,sou,BRE,nor,TOT,eve,MUN,cry,BHA,bur,LIV,wol,WAT,lee,NEW,whu,AVL
2,MUN,LEE,sou,wol,NEW,whu,AVL,EVE,lei,LIV,tot,MCI,wat,che,ARS,CRY,nor,bre,BHA,new,BUR,WOL,avl,WHU,bur,SOU,lee,WAT,mci,TOT,liv,LEI,eve,NOR,ars,BRE,bha,CHE,cry


Selected: AVL and CHE and WOL. Total FDR: 2395.973


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,AVL,0.,58.347,57.676,0.,62.212,0.,0.,59.914,0.,65.414,0.,67.746,71.511,0.,67.325,0.,0.,57.487,0.,0.,0.,73.616,0.,64.501,0.,51.195,0.,57.676,0.,72.058,0.,67.187,76.749,0.,55.972,0.,52.977,0.
1,CHE,52.977,0.,0.,62.126,0.,79.547,57.676,0.,55.972,0.,57.487,0.,0.,69.106,0.,64.501,62.212,0.,0.,67.746,76.749,0.,67.187,0.,72.058,0.,67.325,0.,58.347,0.,57.676,0.,0.,65.414,0.,59.914,0.,51.195
2,WOL,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,51.195,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,AVL,wat,NEW,BRE,che,EVE,mun,tot,WOL,ars,WHU,sou,BHA,cry,MCI,LEI,liv,nor,BUR,CHE,lee,bre,MUN,eve,LEE,new,WAT,bha,SOU,whu,ARS,wol,TOT,LIV,lei,NOR,bur,CRY,mci
1,CHE,CRY,ars,liv,AVL,tot,MCI,SOU,bre,NOR,new,BUR,lei,MUN,wat,whu,LEE,EVE,wol,avl,BHA,LIV,mci,TOT,bha,ARS,cry,LEI,bur,NEW,nor,BRE,sou,lee,WHU,eve,WOL,mun,WAT
2,WOL,lei,TOT,MUN,wat,BRE,sou,NEW,avl,lee,EVE,cry,WHU,nor,BUR,LIV,mci,bha,CHE,WAT,ars,mun,SOU,bre,ARS,tot,LEI,whu,CRY,eve,LEE,AVL,new,MCI,bur,BHA,che,NOR,liv


Selected: ARS and BHA and TOT. Total FDR: 2396.866


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,77.855,0.,0.,55.972,77.599,67.187,0.,52.977,62.126,0.,51.195,0.,58.347,0.,0.,57.676,65.414,0.,0.,59.914,0.,0.,57.487,0.,0.,57.676,76.749,0.,67.325,0.,0.,67.746,0.,73.616,0.,64.501,0.,62.212
1,BHA,0.,51.195,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.
2,TOT,0.,0.,51.195,0.,0.,0.,62.126,0.,0.,73.616,0.,64.501,0.,57.676,55.972,0.,0.,76.749,52.977,0.,69.106,72.058,0.,57.676,59.914,0.,0.,62.212,0.,65.414,58.347,0.,67.746,0.,67.325,0.,57.487,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,bre,CHE,mci,NOR,bur,TOT,bha,CRY,AVL,lei,WAT,liv,NEW,mun,eve,SOU,WHU,lee,nor,WOL,MCI,tot,BUR,wol,che,BRE,LIV,wat,LEI,avl,cry,BHA,sou,MUN,whu,LEE,new,EVE
1,BHA,bur,WAT,EVE,bre,LEI,cry,ARS,nor,MCI,liv,NEW,avl,LEE,whu,sou,TOT,WOL,mun,BRE,che,eve,CRY,lei,CHE,wat,BUR,AVL,new,LIV,mci,NOR,ars,tot,SOU,wol,MUN,lee,WHU
2,TOT,MCI,wol,WAT,cry,CHE,ars,AVL,new,whu,MUN,eve,LEE,bur,BRE,NOR,bha,lei,LIV,CRY,sou,wat,ARS,che,SOU,WOL,mci,lee,EVE,mun,WHU,NEW,avl,BHA,bre,LEI,liv,BUR,nor


In [15]:
solve_N_pair_problem(N=2, max_iter=3, first_gw=1, last_gw=10)

NOTE: Initialized model N_rotation_pairs.

Selected: CHE and MCI. Total FDR: 604.467


,team,1,2,3,4,5,6,7,8,9,10
0,CHE,52.977,0.,0.,62.126,0.,79.547,57.676,0.,55.972,0.
1,MCI,0.,55.972,72.058,0.,57.676,0.,0.,57.487,0.,52.977


,team,1,2,3,4,5,6,7,8,9,10
0,CHE,CRY,ars,liv,AVL,tot,MCI,SOU,bre,NOR,new
1,MCI,tot,NOR,ARS,lei,SOU,che,liv,BUR,bha,CRY



Selected: EVE and WOL. Total FDR: 606.782


,team,1,2,3,4,5,6,7,8,9,10
0,EVE,57.676,0.,0.,57.487,0.,55.972,0.,65.414,51.195,0.
1,WOL,0.,67.187,73.616,0.,57.676,0.,58.347,0.,0.,62.212


,team,1,2,3,4,5,6,7,8,9,10
0,EVE,SOU,lee,bha,BUR,avl,NOR,mun,WHU,WAT,wol
1,WOL,lei,TOT,MUN,wat,BRE,sou,NEW,avl,lee,EVE



Selected: AVL and CHE. Total FDR: 611.86


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,0.,58.347,57.676,0.,62.212,0.,0.,59.914,0.,65.414
1,CHE,52.977,0.,0.,62.126,0.,79.547,57.676,0.,55.972,0.


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,wat,NEW,BRE,che,EVE,mun,tot,WOL,ars,WHU
1,CHE,CRY,ars,liv,AVL,tot,MCI,SOU,bre,NOR,new


In [16]:
def solve_N_pick_K_pair_problem(N=3, K=2, max_iter=1, first_gw=1, last_gw=38, exclude=[]):
    m = so.Model(name='N_K_rotation_pairs')
    team_list = list(teams.keys())
    gameweeks = list(range(first_gw, last_gw+1))
    pick_team = m.add_variables(team_list, vartype=so.binary, name='pick_team')
    pick_team_gw = m.add_variables(team_list, gameweeks, vartype=so.binary, name='pick_team_gw')

    if len(exclude) > 0:
        m.add_constraints((pick_team[t] == 0 for t in exclude), name=f'disable_{t}')

    m.add_constraint(so.expr_sum(pick_team[t] for t in team_list) == N, name='pick_2')
    m.add_constraints((so.expr_sum(pick_team_gw[t, g] for t in team_list) == K for g in gameweeks), name='pick_1_per_gw')
    m.add_constraints((pick_team_gw[t,g] <= pick_team[t] for t in team_list for g in gameweeks), name='valid_picks_only')

    # Force using each team at least once
    m.add_constraints((so.expr_sum(pick_team_gw[t,g] for g in gameweeks) >= pick_team[t] for t in team_list), name='force_use')

    m.set_objective(so.expr_sum(fdr[t, g] * pick_team_gw[t, g] for t in team_list for g in gameweeks), sense='N', name='total_fdr')

    m.export_mps("fdr.mps")
    command = "cbc fdr.mps solve solu fdr.sol"
    Popen(command).wait()
    read_solution(m)
    selected_teams = print_solution(m, gameweeks)
    for it in range(1, max_iter):
        c = m.add_constraint(so.expr_sum(pick_team[t] for t in selected_teams) <= N-1, name=f'cutoff_{it}')
        m.export_mps("fdr.mps")
        Popen(command).wait()
        read_solution(m)
        selected_teams = print_solution(m, gameweeks)

In [17]:
solve_N_pick_K_pair_problem(first_gw=1, last_gw=38, max_iter=3)

NOTE: Initialized model N_K_rotation_pairs.

Selected: CHE and LIV and WHU. Total FDR: 5108.874


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,CHE,52.977,0.,0.,62.126,0.,79.547,57.676,77.855,55.972,78.761,57.487,0.,73.616,69.106,0.,64.501,62.212,80.875,0.,67.746,76.749,0.,67.187,0.,72.058,0.,67.325,77.599,58.347,75.554,57.676,77.855,0.,65.414,0.,59.914,0.,51.195
1,LIV,75.554,57.487,77.920,0.,52.977,77.855,0.,69.106,0.,67.746,0.,72.058,57.676,0.,80.875,62.126,58.347,0.,64.501,0.,0.,57.676,71.511,67.325,77.599,55.972,0.,65.414,0.,73.616,51.195,0.,83.861,62.212,78.761,67.187,77.855,59.914
2,WHU,0.,67.325,52.977,77.855,73.616,0.,57.676,0.,67.187,0.,76.749,80.875,0.,67.746,77.920,0.,0.,55.972,57.676,69.106,71.511,64.501,0.,51.195,0.,58.347,59.914,0.,62.126,0.,0.,77.855,57.487,0.,72.058,0.,79.547,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,CHE,CRY,ars,liv,AVL,tot,MCI,SOU,bre,NOR,new,BUR,lei,MUN,wat,whu,LEE,EVE,wol,avl,BHA,LIV,mci,TOT,bha,ARS,cry,LEI,bur,NEW,nor,BRE,sou,lee,WHU,eve,WOL,mun,WAT
1,LIV,nor,BUR,CHE,lee,CRY,bre,MCI,wat,mun,BHA,whu,ARS,SOU,eve,wol,AVL,NEW,tot,LEE,lei,che,BRE,cry,LEI,bur,NOR,ars,WHU,bha,MUN,WAT,mci,avl,EVE,new,TOT,sou,WOL
2,WHU,new,LEI,CRY,sou,MUN,lee,BRE,eve,TOT,avl,LIV,wol,mci,BHA,CHE,bur,ars,NOR,SOU,wat,cry,LEE,mun,WAT,lei,NEW,WOL,liv,AVL,tot,EVE,bre,BUR,che,ARS,nor,MCI,bha



Selected: AVL and CHE and MUN. Total FDR: 5115.237


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,AVL,0.,58.347,57.676,0.,62.212,0.,0.,59.914,0.,65.414,77.855,67.746,71.511,0.,67.325,0.,75.554,57.487,77.920,0.,0.,73.616,0.,64.501,0.,51.195,0.,57.676,0.,72.058,0.,67.187,76.749,90.879,55.972,77.599,52.977,0.
1,CHE,52.977,0.,0.,62.126,0.,79.547,57.676,77.855,55.972,78.761,57.487,0.,73.616,69.106,0.,64.501,62.212,0.,0.,67.746,76.749,0.,67.187,0.,72.058,71.511,67.325,77.599,58.347,75.554,57.676,77.855,0.,65.414,0.,59.914,0.,51.195
2,MUN,64.501,77.855,80.875,58.347,88.299,62.126,62.212,0.,76.749,0.,0.,69.106,0.,72.058,52.977,75.554,0.,67.746,78.761,57.487,59.914,83.861,65.414,77.599,57.676,0.,51.195,0.,67.187,0.,67.325,0.,55.972,0.,57.676,0.,77.920,71.511


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,AVL,wat,NEW,BRE,che,EVE,mun,tot,WOL,ars,WHU,sou,BHA,cry,MCI,LEI,liv,nor,BUR,CHE,lee,bre,MUN,eve,LEE,new,WAT,bha,SOU,whu,ARS,wol,TOT,LIV,lei,NOR,bur,CRY,mci
1,CHE,CRY,ars,liv,AVL,tot,MCI,SOU,bre,NOR,new,BUR,lei,MUN,wat,whu,LEE,EVE,wol,avl,BHA,LIV,mci,TOT,bha,ARS,cry,LEI,bur,NEW,nor,BRE,sou,lee,WHU,eve,WOL,mun,WAT
2,MUN,LEE,sou,wol,NEW,whu,AVL,EVE,lei,LIV,tot,MCI,wat,che,ARS,CRY,nor,bre,BHA,new,BUR,WOL,avl,WHU,bur,SOU,lee,WAT,mci,TOT,liv,LEI,eve,NOR,ars,BRE,bha,CHE,cry



Selected: CHE and MCI and WHU. Total FDR: 5116.386


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,CHE,52.977,0.,0.,62.126,0.,79.547,57.676,77.855,55.972,78.761,57.487,0.,73.616,69.106,0.,64.501,62.212,0.,0.,67.746,76.749,0.,67.187,0.,72.058,0.,67.325,77.599,58.347,75.554,57.676,77.855,0.,65.414,83.977,59.914,0.,51.195
1,MCI,0.,55.972,72.058,0.,57.676,0.,0.,57.487,0.,52.977,0.,62.212,65.414,0.,69.106,59.914,64.501,78.761,67.325,0.,0.,77.920,77.855,57.676,75.554,67.187,0.,73.616,0.,67.746,0.,76.749,80.875,51.195,0.,58.347,88.299,62.126
2,WHU,78.761,67.325,52.977,77.855,73.616,87.068,57.676,0.,67.187,0.,76.749,80.875,0.,67.746,77.920,0.,0.,55.972,57.676,69.106,71.511,64.501,0.,51.195,0.,58.347,59.914,0.,62.126,0.,62.212,0.,57.487,0.,72.058,0.,79.547,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,CHE,CRY,ars,liv,AVL,tot,MCI,SOU,bre,NOR,new,BUR,lei,MUN,wat,whu,LEE,EVE,wol,avl,BHA,LIV,mci,TOT,bha,ARS,cry,LEI,bur,NEW,nor,BRE,sou,lee,WHU,eve,WOL,mun,WAT
1,MCI,tot,NOR,ARS,lei,SOU,che,liv,BUR,bha,CRY,mun,EVE,WHU,avl,wat,WOL,LEE,new,LEI,bre,ars,CHE,sou,BRE,nor,TOT,eve,MUN,cry,BHA,bur,LIV,wol,WAT,lee,NEW,whu,AVL
2,WHU,new,LEI,CRY,sou,MUN,lee,BRE,eve,TOT,avl,LIV,wol,mci,BHA,CHE,bur,ars,NOR,SOU,wat,cry,LEE,mun,WAT,lei,NEW,WOL,liv,AVL,tot,EVE,bre,BUR,che,ARS,nor,MCI,bha


In [18]:
solve_N_pick_K_pair_problem(first_gw=1, last_gw=10, max_iter=3)

NOTE: Initialized model N_K_rotation_pairs.

Selected: AVL and EVE and LEE. Total FDR: 1298.503


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,69.106,58.347,57.676,0.,62.212,0.,90.693,59.914,0.,65.414
1,EVE,57.676,0.,0.,57.487,0.,55.972,0.,65.414,51.195,0.
2,LEE,0.,62.212,77.599,76.749,78.761,65.414,51.195,0.,59.914,75.554


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,wat,NEW,BRE,che,EVE,mun,tot,WOL,ars,WHU
1,EVE,SOU,lee,bha,BUR,avl,NOR,mun,WHU,WAT,wol
2,LEE,mun,EVE,bur,LIV,new,WHU,WAT,sou,WOL,nor



Selected: AVL and EVE and WOL. Total FDR: 1304.172


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,69.106,58.347,57.676,0.,62.212,0.,90.693,59.914,0.,65.414
1,EVE,57.676,0.,0.,57.487,0.,55.972,0.,65.414,51.195,0.
2,WOL,0.,67.187,73.616,69.106,57.676,77.855,58.347,0.,87.068,62.212


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,wat,NEW,BRE,che,EVE,mun,tot,WOL,ars,WHU
1,EVE,SOU,lee,bha,BUR,avl,NOR,mun,WHU,WAT,wol
2,WOL,lei,TOT,MUN,wat,BRE,sou,NEW,avl,lee,EVE



Selected: AVL and CHE and LEE. Total FDR: 1306.218


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,69.106,58.347,57.676,0.,62.212,0.,0.,59.914,0.,65.414
1,CHE,52.977,0.,0.,62.126,0.,79.547,57.676,77.855,55.972,0.
2,LEE,0.,62.212,77.599,76.749,78.761,65.414,51.195,0.,59.914,75.554


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,wat,NEW,BRE,che,EVE,mun,tot,WOL,ars,WHU
1,CHE,CRY,ars,liv,AVL,tot,MCI,SOU,bre,NOR,new
2,LEE,mun,EVE,bur,LIV,new,WHU,WAT,sou,WOL,nor


In [19]:
solve_N_pick_K_pair_problem(N=4, K=3, first_gw=1, last_gw=38, max_iter=3)


NOTE: Initialized model N_K_rotation_pairs.

Selected: CHE and MCI and MUN and WOL. Total FDR: 7857.592


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,CHE,52.977,0.,0.,62.126,0.,79.547,57.676,77.855,55.972,78.761,57.487,0.,73.616,69.106,0.,64.501,62.212,0.,0.,67.746,76.749,0.,67.187,0.,72.058,71.511,67.325,77.599,58.347,75.554,57.676,77.855,0.,65.414,83.977,59.914,0.,51.195
1,MCI,90.693,55.972,72.058,0.,57.676,0.,0.,57.487,0.,52.977,0.,62.212,65.414,0.,69.106,59.914,64.501,78.761,67.325,77.855,97.269,77.920,0.,57.676,75.554,67.187,83.977,73.616,71.511,67.746,0.,76.749,80.875,51.195,0.,58.347,88.299,62.126
2,MUN,64.501,77.855,80.875,58.347,88.299,62.126,62.212,0.,76.749,0.,79.547,69.106,0.,72.058,52.977,75.554,77.855,67.746,78.761,57.487,59.914,83.861,65.414,77.599,57.676,0.,51.195,0.,67.187,0.,67.325,0.,55.972,0.,57.676,91.448,77.920,71.511
3,WOL,0.,67.187,73.616,69.106,57.676,77.855,58.347,83.861,87.068,62.212,71.511,65.414,75.554,57.487,76.749,0.,0.,77.920,51.195,0.,0.,57.676,77.855,72.058,0.,67.325,0.,52.977,0.,64.501,62.126,78.761,79.547,77.599,67.746,0.,55.972,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,CHE,CRY,ars,liv,AVL,tot,MCI,SOU,bre,NOR,new,BUR,lei,MUN,wat,whu,LEE,EVE,wol,avl,BHA,LIV,mci,TOT,bha,ARS,cry,LEI,bur,NEW,nor,BRE,sou,lee,WHU,eve,WOL,mun,WAT
1,MCI,tot,NOR,ARS,lei,SOU,che,liv,BUR,bha,CRY,mun,EVE,WHU,avl,wat,WOL,LEE,new,LEI,bre,ars,CHE,sou,BRE,nor,TOT,eve,MUN,cry,BHA,bur,LIV,wol,WAT,lee,NEW,whu,AVL
2,MUN,LEE,sou,wol,NEW,whu,AVL,EVE,lei,LIV,tot,MCI,wat,che,ARS,CRY,nor,bre,BHA,new,BUR,WOL,avl,WHU,bur,SOU,lee,WAT,mci,TOT,liv,LEI,eve,NOR,ars,BRE,bha,CHE,cry
3,WOL,lei,TOT,MUN,wat,BRE,sou,NEW,avl,lee,EVE,cry,WHU,nor,BUR,LIV,mci,bha,CHE,WAT,ars,mun,SOU,bre,ARS,tot,LEI,whu,CRY,eve,LEE,AVL,new,MCI,bur,BHA,che,NOR,liv



Selected: CHE and LIV and MCI and WHU. Total FDR: 7860.275


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,CHE,52.977,0.,0.,62.126,0.,79.547,57.676,77.855,55.972,78.761,57.487,0.,73.616,69.106,0.,64.501,62.212,80.875,0.,67.746,76.749,0.,67.187,0.,72.058,0.,67.325,77.599,58.347,75.554,57.676,77.855,0.,65.414,83.977,59.914,0.,51.195
1,LIV,75.554,57.487,77.920,87.068,52.977,77.855,79.547,69.106,0.,67.746,88.299,72.058,57.676,0.,80.875,62.126,58.347,0.,64.501,0.,0.,57.676,71.511,67.325,77.599,55.972,0.,65.414,0.,73.616,51.195,0.,83.861,62.212,78.761,67.187,77.855,59.914
2,MCI,0.,55.972,72.058,0.,57.676,0.,0.,57.487,91.448,52.977,0.,62.212,65.414,83.861,69.106,59.914,64.501,78.761,67.325,77.855,97.269,77.920,77.855,57.676,75.554,67.187,83.977,73.616,71.511,67.746,0.,76.749,80.875,51.195,0.,58.347,88.299,62.126
3,WHU,78.761,67.325,52.977,77.855,73.616,87.068,57.676,0.,67.187,0.,76.749,80.875,0.,67.746,77.920,0.,0.,55.972,57.676,69.106,71.511,64.501,0.,51.195,0.,58.347,59.914,0.,62.126,0.,62.212,77.855,57.487,0.,72.058,0.,79.547,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,CHE,CRY,ars,liv,AVL,tot,MCI,SOU,bre,NOR,new,BUR,lei,MUN,wat,whu,LEE,EVE,wol,avl,BHA,LIV,mci,TOT,bha,ARS,cry,LEI,bur,NEW,nor,BRE,sou,lee,WHU,eve,WOL,mun,WAT
1,LIV,nor,BUR,CHE,lee,CRY,bre,MCI,wat,mun,BHA,whu,ARS,SOU,eve,wol,AVL,NEW,tot,LEE,lei,che,BRE,cry,LEI,bur,NOR,ars,WHU,bha,MUN,WAT,mci,avl,EVE,new,TOT,sou,WOL
2,MCI,tot,NOR,ARS,lei,SOU,che,liv,BUR,bha,CRY,mun,EVE,WHU,avl,wat,WOL,LEE,new,LEI,bre,ars,CHE,sou,BRE,nor,TOT,eve,MUN,cry,BHA,bur,LIV,wol,WAT,lee,NEW,whu,AVL
3,WHU,new,LEI,CRY,sou,MUN,lee,BRE,eve,TOT,avl,LIV,wol,mci,BHA,CHE,bur,ars,NOR,SOU,wat,cry,LEE,mun,WAT,lei,NEW,WOL,liv,AVL,tot,EVE,bre,BUR,che,ARS,nor,MCI,bha



Selected: CHE and MCI and MUN and TOT. Total FDR: 7861.67


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,CHE,52.977,0.,0.,62.126,0.,79.547,57.676,77.855,55.972,78.761,57.487,0.,73.616,69.106,0.,64.501,62.212,0.,0.,67.746,76.749,0.,67.187,0.,72.058,71.511,67.325,77.599,58.347,75.554,57.676,77.855,0.,65.414,83.977,59.914,0.,51.195
1,MCI,0.,55.972,72.058,0.,57.676,0.,0.,57.487,0.,52.977,0.,62.212,65.414,0.,69.106,59.914,64.501,78.761,67.325,0.,0.,77.920,77.855,57.676,0.,67.187,83.977,73.616,71.511,67.746,0.,76.749,80.875,51.195,0.,58.347,88.299,62.126
2,MUN,64.501,77.855,80.875,58.347,88.299,62.126,62.212,0.,76.749,0.,79.547,69.106,0.,72.058,52.977,75.554,77.855,67.746,78.761,57.487,59.914,83.861,65.414,77.599,57.676,87.068,51.195,0.,67.187,0.,67.325,0.,55.972,0.,57.676,91.448,77.920,71.511
3,TOT,79.547,80.875,51.195,71.511,77.920,97.269,62.126,78.761,88.299,73.616,83.977,64.501,77.599,57.676,55.972,0.,0.,76.749,52.977,77.855,69.106,72.058,0.,57.676,59.914,0.,0.,62.212,0.,65.414,58.347,83.861,67.746,77.855,67.325,0.,57.487,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,CHE,CRY,ars,liv,AVL,tot,MCI,SOU,bre,NOR,new,BUR,lei,MUN,wat,whu,LEE,EVE,wol,avl,BHA,LIV,mci,TOT,bha,ARS,cry,LEI,bur,NEW,nor,BRE,sou,lee,WHU,eve,WOL,mun,WAT
1,MCI,tot,NOR,ARS,lei,SOU,che,liv,BUR,bha,CRY,mun,EVE,WHU,avl,wat,WOL,LEE,new,LEI,bre,ars,CHE,sou,BRE,nor,TOT,eve,MUN,cry,BHA,bur,LIV,wol,WAT,lee,NEW,whu,AVL
2,MUN,LEE,sou,wol,NEW,whu,AVL,EVE,lei,LIV,tot,MCI,wat,che,ARS,CRY,nor,bre,BHA,new,BUR,WOL,avl,WHU,bur,SOU,lee,WAT,mci,TOT,liv,LEI,eve,NOR,ars,BRE,bha,CHE,cry
3,TOT,MCI,wol,WAT,cry,CHE,ars,AVL,new,whu,MUN,eve,LEE,bur,BRE,NOR,bha,lei,LIV,CRY,sou,wat,ARS,che,SOU,WOL,mci,lee,EVE,mun,WHU,NEW,avl,BHA,bre,LEI,liv,BUR,nor


In [20]:
solve_N_pick_K_pair_problem(first_gw=1, last_gw=38, max_iter=3, exclude=['MCI', 'CHE', 'LIV', 'MUN'])

NOTE: Initialized model N_K_rotation_pairs.

Selected: BHA and SOU and TOT. Total FDR: 5130.147


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,BHA,77.599,51.195,62.212,0.,67.325,71.511,72.058,75.554,79.547,0.,58.347,0.,64.501,0.,0.,67.187,59.914,0.,57.676,0.,0.,52.977,90.879,77.920,69.106,57.487,62.126,78.761,76.749,0.,55.972,0.,0.,57.676,0.,73.616,0.,65.414
1,SOU,0.,73.616,0.,65.414,0.,59.914,0.,64.501,57.487,69.106,62.126,75.554,0.,67.325,67.746,0.,71.511,57.676,0.,67.187,58.347,0.,79.547,0.,0.,62.212,55.972,0.,51.195,77.599,0.,77.920,72.058,0.,52.977,77.855,76.749,0.
2,TOT,79.547,0.,51.195,71.511,77.920,0.,62.126,0.,0.,73.616,0.,64.501,77.599,57.676,55.972,91.448,0.,76.749,52.977,77.855,69.106,72.058,0.,57.676,59.914,0.,0.,62.212,0.,65.414,58.347,83.861,67.746,77.855,67.325,0.,57.487,75.554


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,BHA,bur,WAT,EVE,bre,LEI,cry,ARS,nor,MCI,liv,NEW,avl,LEE,whu,sou,TOT,WOL,mun,BRE,che,eve,CRY,lei,CHE,wat,BUR,AVL,new,LIV,mci,NOR,ars,tot,SOU,wol,MUN,lee,WHU
1,SOU,eve,MUN,new,WHU,mci,WOL,che,LEE,BUR,wat,AVL,nor,liv,LEI,BHA,ars,cry,BRE,whu,TOT,NEW,wol,MCI,tot,mun,EVE,NOR,avl,WAT,bur,lee,CHE,ARS,bha,CRY,bre,LIV,lei
2,TOT,MCI,wol,WAT,cry,CHE,ars,AVL,new,whu,MUN,eve,LEE,bur,BRE,NOR,bha,lei,LIV,CRY,sou,wat,ARS,che,SOU,WOL,mci,lee,EVE,mun,WHU,NEW,avl,BHA,bre,LEI,liv,BUR,nor



Selected: ARS and LEI and WHU. Total FDR: 5137.574


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,77.855,77.920,0.,55.972,77.599,67.187,0.,52.977,62.126,0.,51.195,0.,58.347,0.,0.,57.676,65.414,0.,75.554,59.914,0.,0.,57.487,80.875,0.,57.676,76.749,69.106,67.325,83.861,71.511,67.746,77.855,73.616,88.299,64.501,78.761,62.212
1,LEI,59.914,0.,75.554,0.,0.,57.487,71.511,73.616,0.,72.058,0.,77.920,51.195,77.855,83.861,58.347,67.187,83.977,0.,0.,55.972,77.599,67.746,0.,65.414,0.,0.,64.501,0.,57.676,0.,52.977,0.,62.126,0.,62.212,69.106,57.676
2,WHU,0.,67.325,52.977,77.855,73.616,0.,57.676,0.,67.187,83.861,76.749,80.875,0.,67.746,77.920,0.,0.,55.972,57.676,69.106,71.511,64.501,0.,51.195,90.879,58.347,59.914,0.,62.126,0.,62.212,0.,57.487,0.,72.058,0.,0.,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,ARS,bre,CHE,mci,NOR,bur,TOT,bha,CRY,AVL,lei,WAT,liv,NEW,mun,eve,SOU,WHU,lee,nor,WOL,MCI,tot,BUR,wol,che,BRE,LIV,wat,LEI,avl,cry,BHA,sou,MUN,whu,LEE,new,EVE
1,LEI,WOL,whu,nor,MCI,bha,BUR,cry,MUN,bre,ARS,lee,CHE,WAT,sou,avl,NEW,TOT,eve,mci,LIV,NOR,bur,BHA,liv,WHU,wol,che,LEE,ars,BRE,mun,CRY,new,AVL,tot,EVE,wat,SOU
2,WHU,new,LEI,CRY,sou,MUN,lee,BRE,eve,TOT,avl,LIV,wol,mci,BHA,CHE,bur,ars,NOR,SOU,wat,cry,LEE,mun,WAT,lei,NEW,WOL,liv,AVL,tot,EVE,bre,BUR,che,ARS,nor,MCI,bha



Selected: BHA and LEI and WHU. Total FDR: 5142.111


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,BHA,77.599,51.195,62.212,77.855,67.325,71.511,0.,75.554,0.,0.,58.347,0.,64.501,0.,77.855,67.187,59.914,0.,57.676,0.,0.,52.977,90.879,77.920,69.106,57.487,62.126,78.761,76.749,0.,55.972,0.,0.,57.676,80.875,73.616,0.,65.414
1,LEI,59.914,0.,0.,0.,0.,57.487,71.511,73.616,77.855,72.058,0.,77.920,51.195,77.855,0.,58.347,67.187,83.977,0.,76.749,55.972,0.,67.746,0.,65.414,0.,0.,64.501,0.,57.676,0.,52.977,78.761,62.126,0.,62.212,69.106,57.676
2,WHU,0.,67.325,52.977,77.855,73.616,0.,57.676,0.,67.187,83.861,76.749,80.875,0.,67.746,77.920,0.,0.,55.972,57.676,69.106,71.511,64.501,0.,51.195,0.,58.347,59.914,0.,62.126,90.693,62.212,77.855,57.487,0.,72.058,0.,79.547,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,BHA,bur,WAT,EVE,bre,LEI,cry,ARS,nor,MCI,liv,NEW,avl,LEE,whu,sou,TOT,WOL,mun,BRE,che,eve,CRY,lei,CHE,wat,BUR,AVL,new,LIV,mci,NOR,ars,tot,SOU,wol,MUN,lee,WHU
1,LEI,WOL,whu,nor,MCI,bha,BUR,cry,MUN,bre,ARS,lee,CHE,WAT,sou,avl,NEW,TOT,eve,mci,LIV,NOR,bur,BHA,liv,WHU,wol,che,LEE,ars,BRE,mun,CRY,new,AVL,tot,EVE,wat,SOU
2,WHU,new,LEI,CRY,sou,MUN,lee,BRE,eve,TOT,avl,LIV,wol,mci,BHA,CHE,bur,ars,NOR,SOU,wat,cry,LEE,mun,WAT,lei,NEW,WOL,liv,AVL,tot,EVE,bre,BUR,che,ARS,nor,MCI,bha


In [21]:
solve_N_pick_K_pair_problem(first_gw=1, last_gw=10, max_iter=3, exclude=['MCI', 'CHE', 'LIV', 'MUN'])

NOTE: Initialized model N_K_rotation_pairs.

Selected: AVL and EVE and LEE. Total FDR: 1298.503


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,69.106,58.347,57.676,0.,62.212,0.,90.693,59.914,0.,65.414
1,EVE,57.676,0.,0.,57.487,0.,55.972,0.,65.414,51.195,0.
2,LEE,0.,62.212,77.599,76.749,78.761,65.414,51.195,0.,59.914,75.554


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,wat,NEW,BRE,che,EVE,mun,tot,WOL,ars,WHU
1,EVE,SOU,lee,bha,BUR,avl,NOR,mun,WHU,WAT,wol
2,LEE,mun,EVE,bur,LIV,new,WHU,WAT,sou,WOL,nor



Selected: AVL and EVE and WOL. Total FDR: 1304.172


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,69.106,58.347,57.676,0.,62.212,0.,90.693,59.914,0.,65.414
1,EVE,57.676,0.,0.,57.487,0.,55.972,0.,65.414,51.195,0.
2,WOL,0.,67.187,73.616,69.106,57.676,77.855,58.347,0.,87.068,62.212


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,wat,NEW,BRE,che,EVE,mun,tot,WOL,ars,WHU
1,EVE,SOU,lee,bha,BUR,avl,NOR,mun,WHU,WAT,wol
2,WOL,lei,TOT,MUN,wat,BRE,sou,NEW,avl,lee,EVE



Selected: AVL and BHA and EVE. Total FDR: 1313.682


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,69.106,58.347,57.676,0.,62.212,0.,90.693,59.914,0.,65.414
1,BHA,0.,51.195,62.212,77.855,67.325,71.511,72.058,0.,79.547,0.
2,EVE,57.676,0.,0.,57.487,0.,55.972,0.,65.414,51.195,80.875


,team,1,2,3,4,5,6,7,8,9,10
0,AVL,wat,NEW,BRE,che,EVE,mun,tot,WOL,ars,WHU
1,BHA,bur,WAT,EVE,bre,LEI,cry,ARS,nor,MCI,liv
2,EVE,SOU,lee,bha,BUR,avl,NOR,mun,WHU,WAT,wol


In [22]:
solve_N_pick_K_pair_problem(N=3, K=2, first_gw=1, last_gw=38, max_iter=3, exclude=['MCI', 'CHE', 'LIV', 'MUN', 'LEI', 'ARS', 'TOT'])

NOTE: Initialized model N_K_rotation_pairs.

Selected: BHA and LEE and WAT. Total FDR: 5152.075


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,BHA,77.599,51.195,62.212,0.,67.325,0.,72.058,75.554,79.547,0.,58.347,83.861,64.501,0.,77.855,67.187,59.914,0.,57.676,0.,0.,52.977,0.,77.920,69.106,57.487,62.126,78.761,76.749,0.,55.972,0.,0.,57.676,0.,73.616,0.,65.414
1,LEE,0.,62.212,77.599,76.749,0.,65.414,51.195,0.,59.914,75.554,67.325,0.,0.,52.977,57.676,0.,0.,72.058,0.,62.126,57.487,0.,58.347,83.861,0.,73.616,67.187,0.,55.972,80.875,57.676,69.106,77.920,71.511,79.547,0.,67.746,77.855
2,WAT,62.126,0.,0.,59.914,75.554,58.347,0.,76.749,0.,57.676,0.,73.616,90.879,77.920,0.,77.855,77.599,52.977,80.875,65.414,67.187,78.761,55.972,0.,67.746,0.,0.,72.058,0.,62.212,0.,64.501,57.676,0.,57.487,71.511,67.325,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,BHA,bur,WAT,EVE,bre,LEI,cry,ARS,nor,MCI,liv,NEW,avl,LEE,whu,sou,TOT,WOL,mun,BRE,che,eve,CRY,lei,CHE,wat,BUR,AVL,new,LIV,mci,NOR,ars,tot,SOU,wol,MUN,lee,WHU
1,LEE,mun,EVE,bur,LIV,new,WHU,WAT,sou,WOL,nor,LEI,tot,bha,CRY,BRE,che,mci,ARS,liv,AVL,BUR,whu,NEW,avl,eve,MUN,TOT,lei,NOR,wol,SOU,wat,CHE,cry,MCI,ars,BHA,bre
2,WAT,AVL,bha,tot,WOL,nor,NEW,lee,LIV,eve,SOU,ars,MUN,lei,CHE,MCI,bre,bur,CRY,wol,WHU,TOT,new,NOR,whu,BHA,avl,mun,ARS,sou,EVE,liv,LEE,BRE,mci,BUR,cry,LEI,che



Selected: BHA and LEE and NOR. Total FDR: 5156.449


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,BHA,77.599,51.195,62.212,77.855,67.325,71.511,72.058,75.554,79.547,0.,58.347,83.861,64.501,0.,77.855,67.187,59.914,0.,57.676,0.,83.977,52.977,0.,77.920,69.106,57.487,62.126,78.761,76.749,0.,55.972,0.,90.693,57.676,80.875,73.616,0.,65.414
1,LEE,0.,62.212,0.,76.749,0.,65.414,51.195,0.,59.914,75.554,67.325,0.,0.,52.977,57.676,0.,0.,72.058,0.,62.126,57.487,0.,58.347,0.,0.,73.616,67.187,0.,55.972,80.875,57.676,69.106,77.920,0.,79.547,0.,67.746,0.
2,NOR,76.749,0.,67.325,0.,51.195,0.,0.,67.746,0.,64.501,0.,57.676,59.914,78.761,0.,73.616,62.126,88.299,72.058,71.511,0.,62.212,69.106,52.977,79.547,0.,0.,57.676,0.,77.920,0.,57.487,0.,58.347,0.,65.414,80.875,67.187


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,BHA,bur,WAT,EVE,bre,LEI,cry,ARS,nor,MCI,liv,NEW,avl,LEE,whu,sou,TOT,WOL,mun,BRE,che,eve,CRY,lei,CHE,wat,BUR,AVL,new,LIV,mci,NOR,ars,tot,SOU,wol,MUN,lee,WHU
1,LEE,mun,EVE,bur,LIV,new,WHU,WAT,sou,WOL,nor,LEI,tot,bha,CRY,BRE,che,mci,ARS,liv,AVL,BUR,whu,NEW,avl,eve,MUN,TOT,lei,NOR,wol,SOU,wat,CHE,cry,MCI,ars,BHA,bre
2,NOR,LIV,mci,LEI,ars,WAT,eve,bur,BHA,che,LEE,bre,SOU,WOL,new,tot,MUN,AVL,whu,ARS,cry,lei,EVE,wat,CRY,MCI,liv,sou,BRE,lee,CHE,bha,BUR,mun,NEW,avl,WHU,wol,TOT



Selected: BHA and NOR and SOU. Total FDR: 5165.249


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,BHA,77.599,51.195,62.212,77.855,67.325,71.511,72.058,0.,79.547,0.,58.347,0.,64.501,0.,77.855,67.187,59.914,0.,57.676,0.,83.977,52.977,0.,77.920,69.106,57.487,62.126,78.761,76.749,0.,55.972,0.,90.693,57.676,80.875,73.616,0.,65.414
1,NOR,76.749,0.,67.325,0.,51.195,0.,77.599,67.746,0.,64.501,0.,57.676,59.914,78.761,0.,73.616,62.126,88.299,72.058,71.511,0.,62.212,69.106,52.977,79.547,0.,0.,57.676,0.,77.920,0.,57.487,0.,58.347,0.,65.414,80.875,67.187
2,SOU,0.,73.616,0.,65.414,0.,59.914,0.,64.501,57.487,69.106,62.126,75.554,0.,67.325,67.746,0.,0.,57.676,0.,67.187,58.347,0.,79.547,0.,0.,62.212,55.972,0.,51.195,77.599,87.068,77.920,72.058,0.,52.977,0.,76.749,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,BHA,bur,WAT,EVE,bre,LEI,cry,ARS,nor,MCI,liv,NEW,avl,LEE,whu,sou,TOT,WOL,mun,BRE,che,eve,CRY,lei,CHE,wat,BUR,AVL,new,LIV,mci,NOR,ars,tot,SOU,wol,MUN,lee,WHU
1,NOR,LIV,mci,LEI,ars,WAT,eve,bur,BHA,che,LEE,bre,SOU,WOL,new,tot,MUN,AVL,whu,ARS,cry,lei,EVE,wat,CRY,MCI,liv,sou,BRE,lee,CHE,bha,BUR,mun,NEW,avl,WHU,wol,TOT
2,SOU,eve,MUN,new,WHU,mci,WOL,che,LEE,BUR,wat,AVL,nor,liv,LEI,BHA,ars,cry,BRE,whu,TOT,NEW,wol,MCI,tot,mun,EVE,NOR,avl,WAT,bur,lee,CHE,ARS,bha,CRY,bre,LIV,lei


In [25]:
solve_N_pair_problem(N=2, first_gw=1, last_gw=19, max_iter=3)

NOTE: Initialized model N_rotation_pairs.

Selected: BUR and SOU. Total FDR: 1192.743


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,BUR,67.746,0.,64.501,0.,72.058,0.,55.972,0.,0.,57.676,0.,52.977,67.187,0.,0.,65.414,51.195,0.,62.212
1,SOU,0.,73.616,0.,65.414,0.,59.914,0.,64.501,57.487,0.,62.126,0.,0.,67.325,67.746,0.,0.,57.676,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,BUR,BHA,liv,LEE,eve,ARS,lei,NOR,mci,sou,BRE,che,CRY,TOT,wol,new,WHU,WAT,avl,EVE
1,SOU,eve,MUN,new,WHU,mci,WOL,che,LEE,BUR,wat,AVL,nor,liv,LEI,BHA,ars,cry,BRE,whu



Selected: CHE and MCI. Total FDR: 1196.002


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,CHE,52.977,0.,0.,62.126,0.,79.547,57.676,0.,55.972,0.,57.487,0.,0.,69.106,0.,0.,62.212,0.,0.
1,MCI,0.,55.972,72.058,0.,57.676,0.,0.,57.487,0.,52.977,0.,62.212,65.414,0.,69.106,59.914,0.,78.761,67.325


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,CHE,CRY,ars,liv,AVL,tot,MCI,SOU,bre,NOR,new,BUR,lei,MUN,wat,whu,LEE,EVE,wol,avl
1,MCI,tot,NOR,ARS,lei,SOU,che,liv,BUR,bha,CRY,mun,EVE,WHU,avl,wat,WOL,LEE,new,LEI



Selected: ARS and WOL. Total FDR: 1196.551


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,ARS,77.855,0.,0.,55.972,0.,67.187,0.,52.977,62.126,0.,51.195,0.,58.347,0.,0.,57.676,65.414,0.,0.
1,WOL,0.,67.187,73.616,0.,57.676,0.,58.347,0.,0.,62.212,0.,65.414,0.,57.487,76.749,0.,0.,77.920,51.195


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,ARS,bre,CHE,mci,NOR,bur,TOT,bha,CRY,AVL,lei,WAT,liv,NEW,mun,eve,SOU,WHU,lee,nor
1,WOL,lei,TOT,MUN,wat,BRE,sou,NEW,avl,lee,EVE,cry,WHU,nor,BUR,LIV,mci,bha,CHE,WAT


In [26]:
solve_N_pair_problem(N=2, first_gw=1, last_gw=19, max_iter=3, exclude=['MCI', 'CHE', 'LIV', 'MUN', 'LEI', 'ARS', 'TOT'])

NOTE: Initialized model N_rotation_pairs.

Selected: BUR and SOU. Total FDR: 1192.743


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,BUR,67.746,0.,64.501,0.,72.058,0.,55.972,0.,0.,57.676,0.,52.977,67.187,0.,0.,65.414,51.195,0.,62.212
1,SOU,0.,73.616,0.,65.414,0.,59.914,0.,64.501,57.487,0.,62.126,0.,0.,67.325,67.746,0.,0.,57.676,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,BUR,BHA,liv,LEE,eve,ARS,lei,NOR,mci,sou,BRE,che,CRY,TOT,wol,new,WHU,WAT,avl,EVE
1,SOU,eve,MUN,new,WHU,mci,WOL,che,LEE,BUR,wat,AVL,nor,liv,LEI,BHA,ars,cry,BRE,whu



Selected: BUR and LEE. Total FDR: 1204.341


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,BUR,67.746,0.,64.501,0.,72.058,0.,0.,0.,0.,57.676,0.,52.977,67.187,0.,0.,65.414,51.195,0.,62.212
1,LEE,0.,62.212,0.,76.749,0.,65.414,51.195,77.855,59.914,0.,67.325,0.,0.,52.977,57.676,0.,0.,72.058,0.


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,BUR,BHA,liv,LEE,eve,ARS,lei,NOR,mci,sou,BRE,che,CRY,TOT,wol,new,WHU,WAT,avl,EVE
1,LEE,mun,EVE,bur,LIV,new,WHU,WAT,sou,WOL,nor,LEI,tot,bha,CRY,BRE,che,mci,ARS,liv



Selected: CRY and NEW. Total FDR: 1206.764


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,CRY,0.,57.676,0.,67.187,0.,67.746,67.325,0.,58.347,0.,59.914,0.,62.126,0.,0.,62.212,57.676,69.106,0.
1,NEW,65.414,0.,57.676,0.,64.501,0.,0.,67.187,0.,77.920,0.,57.676,0.,55.972,57.487,0.,0.,0.,73.616


,team,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,CRY,che,BRE,whu,TOT,liv,BHA,LEI,ars,NEW,mci,WOL,bur,AVL,lee,mun,EVE,SOU,wat,tot
1,NEW,WHU,avl,SOU,mun,LEE,wat,wol,TOT,cry,CHE,bha,BRE,ars,NOR,BUR,lei,liv,MCI,MUN
